In [1]:
from math import pi

import pandas as pd

from bokeh.io import show
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data
from bokeh.palettes import viridis
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import picaso.picaso_data as ptest
import picaso.disort_disco as dtest

/Users/crooney/anaconda3/envs/conda_venv/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /Users/crooney/Documents/codes/all-data/stellar/grp/hst/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/Users/crooney/anaconda3/envs/conda_venv/lib/python3.7/site-packages/pysynphot/refs.py:118: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /Users/crooney/Documents/codes/all-data/stellar/grp/hst/cdbs/mtab/*_tmg.fits
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/crooney/anaconda3/envs/conda_venv/lib/python3.7/site-packages/pysynphot/refs.py:125: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for /Users/crooney/Documents/codes/all-data/stellar/grp/hst/cdbs/mtab/*_tmt.fits
  'no thermal calculations can be performed. ' + str(e))


In [3]:
# # # Toon quadrature
# Toon_quad = ptest.picaso_albedos(output_dir='Toon_quad.csv', disort_data=False)
# # Toon eddington
# Toon_edd = ptest.picaso_albedos(output_dir= 'Toon_eddd.csv',Toon_coefficients ="eddington")
# SH 2-stream
# SH2 = ptest.picaso_albedos(output_dir='SH2_.csv',method="SH", delta_eddington=True, rayleigh=False)
# SH 4-stream
SH4 = ptest.picaso_albedos(output_dir='SH4_.csv',method='SH', stream=4, delta_eddington=True,rayleigh=False)

YOU ARE REQUESTING RESAMPLING!!
NO DATA GENERATED FOR DISORT


In [ ]:
# disort = dtest.disort_albedos(output_dir='disort.csv')

In [5]:
# read in data from picaso function
data_old = pd.read_csv('DLUGACH_TEST.csv',index_col=0)
data_disort = pd.read_csv('disort.csv',index_col=0)
data_SH2 = pd.read_csv('SH2.csv',index_col=0)
data_SH4 = pd.read_csv('SH4.csv',index_col=0)
data_quad = pd.read_csv('Toon_quad.csv',index_col=0)
data_edd = pd.read_csv('Toon_eddd.csv',index_col=0)

#If you want to compare two different files for the heatmap just do this
data_compare = data_old
compare_quad = (data_compare-data_quad)/data_compare*100   
compare_edd = (data_compare-data_edd)/data_compare*100   
compare_SH2 = (data_compare-data_SH2)/data_compare*100 
compare_SH4 = (data_compare-data_SH4)/data_compare*100   

SH4_SH2 = (data_SH4-data_SH2)/data_edd*100
 

In [6]:
from bokeh.palettes import RdGy
def create_heat_map(data):
    data.columns.name = 'g0' 
    data.index.name = 'w0' 
    data.index=data.index.astype(str)
    data = data.rename(index={"-1.0":"Ray"})


    w0s = list(data.index)
    g0s = list(data.columns)

    # reshape to 1D array or rates with a month and year for each row.
    df = pd.DataFrame(data.stack(), columns=['albedo']).reset_index()

    colors = RdGy[11]
    bd = max(abs(df.albedo.min()), abs(df.albedo.max()))
    bd = min(bd,50)
    mapper = LinearColorMapper(palette=colors, low=-bd, high=bd)

    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

    p = figure(height=300,width=300,
           x_range=w0s, y_range=list(reversed(g0s)),
           x_axis_location="above",
           tools=TOOLS, toolbar_location='below')

    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "7px"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = pi / 3

    p.rect(x="w0", y="g0", width=1, height=1,
       source=df,
       fill_color={'field': 'albedo', 'transform': mapper},
       line_color=None)

    color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="12px",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     label_standoff=6, border_line_color=None, location=(0, 0))
    p.add_layout(color_bar, 'right')
    p.axis.major_label_text_font_size='12px'
    return p

In [7]:
show(create_heat_map(compare_quad))

In [8]:
show(create_heat_map(SH4_SH2))

In [9]:
show(create_heat_map(compare_SH2)

SyntaxError: unexpected EOF while parsing (<ipython-input-9-7fae8be883e3>, line 1)

In [10]:
show(create_heat_map(compare_SH4))

In [11]:
data_SH4

g0,1.000,0.999,0.995,0.990,0.980,0.950,0.900,0.800,0.700
w0,,,,,,,,,
-1.00,0.736643,0.684940,0.625190,0.584426,0.531843,0.442005,0.358539,0.263127,0.202493
0.00,0.673223,0.625592,0.566183,0.525729,0.473686,0.385370,0.304490,0.214547,0.159652
0.50,0.631872,0.573845,0.500964,0.452513,0.391765,0.293442,0.209920,0.126786,0.082726
0.75,0.625351,0.555775,0.467642,0.410908,0.342104,0.237063,0.155278,0.082706,0.048952
0.80,0.622861,0.548777,0.454545,0.394745,0.323284,0.216940,0.137196,0.069733,0.039939
0.85,0.618677,0.538333,0.435432,0.371537,0.296870,0.189951,0.114165,0.054362,0.029822
0.90,0.610266,0.520614,0.404215,0.334616,0.256367,0.151459,0.083832,0.036155,0.018692


In [11]:
data_SH4

g0,1.000,0.999,0.995,0.990,0.980,0.950,0.900,0.800,0.700
w0,,,,,,,,,
-1.00,0.736643,0.684940,0.625190,0.584426,0.531843,0.442005,0.358539,0.263127,0.202493
0.00,0.673223,0.625592,0.566183,0.525729,0.473686,0.385370,0.304490,0.214547,0.159652
0.50,0.631872,0.573845,0.500964,0.452513,0.391765,0.293442,0.209920,0.126786,0.082726
0.75,0.625351,0.555775,0.467642,0.410908,0.342104,0.237063,0.155278,0.082706,0.048952
0.80,0.622861,0.548777,0.454545,0.394745,0.323284,0.216940,0.137196,0.069733,0.039939
0.85,0.618677,0.538333,0.435432,0.371537,0.296870,0.189951,0.114165,0.054362,0.029822
0.90,0.610266,0.520614,0.404215,0.334616,0.256367,0.151459,0.083832,0.036155,0.018692
